# EBAC - Regressão II - regressão múltipla

## Tarefa I

#### Previsão de renda II

Vamos continuar trabalhando com a base 'previsao_de_renda.csv', que é a base do seu próximo projeto. Vamos usar os recursos que vimos até aqui nesta base.

|variavel|descrição|
|-|-|
|data_ref                | Data de referência de coleta das variáveis |
|index                   | Código de identificação do cliente|
|sexo                    | Sexo do cliente|
|posse_de_veiculo        | Indica se o cliente possui veículo|
|posse_de_imovel         | Indica se o cliente possui imóvel|
|qtd_filhos              | Quantidade de filhos do cliente|
|tipo_renda              | Tipo de renda do cliente|
|educacao                | Grau de instrução do cliente|
|estado_civil            | Estado civil do cliente|
|tipo_residencia         | Tipo de residência do cliente (própria, alugada etc)|
|idade                   | Idade do cliente|
|tempo_emprego           | Tempo no emprego atual|
|qt_pessoas_residencia   | Quantidade de pessoas que moram na residência|
|renda                   | Renda em reais|

In [47]:
import pandas as pd
import numpy as np
from patsy import dmatrices
import statsmodels.api as sm
from sklearn.model_selection import train_test_split
from sklearn.linear_model import Ridge
from sklearn.metrics import r2_score
from sklearn.linear_model import Lasso
from sklearn.feature_selection import RFE
from sklearn.linear_model import LinearRegression
from sklearn.tree import DecisionTreeRegressor



## 1.

In [2]:
df = pd.read_csv('Profissão- Cientista de Dados_M10_support material.csv')

In [3]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 15000 entries, 0 to 14999
Data columns (total 16 columns):
 #   Column                 Non-Null Count  Dtype  
---  ------                 --------------  -----  
 0   Unnamed: 0             15000 non-null  int64  
 1   data_ref               15000 non-null  object 
 2   index                  15000 non-null  int64  
 3   sexo                   15000 non-null  object 
 4   posse_de_veiculo       15000 non-null  bool   
 5   posse_de_imovel        15000 non-null  bool   
 6   qtd_filhos             15000 non-null  int64  
 7   tipo_renda             15000 non-null  object 
 8   educacao               15000 non-null  object 
 9   estado_civil           15000 non-null  object 
 10  tipo_residencia        15000 non-null  object 
 11  idade                  15000 non-null  int64  
 12  tempo_emprego          12466 non-null  float64
 13  qt_pessoas_residencia  15000 non-null  float64
 14  mau                    15000 non-null  bool   
 15  re

1. Separe a base em treinamento e teste (25% para teste, 75% para treinamento).
2. Rode uma regularização *ridge* com alpha = [0, 0.001, 0.005, 0.01, 0.05, 0.1] e avalie o $R^2$ na base de testes. Qual o melhor modelo?
3. Faça o mesmo que no passo 2, com uma regressão *LASSO*. Qual método chega a um melhor resultado?
4. Rode um modelo *stepwise*. Avalie o $R^2$ na vase de testes. Qual o melhor resultado?
5. Compare os parâmetros e avalie eventuais diferenças. Qual modelo você acha o melhor de todos?
6. Partindo dos modelos que você ajustou, tente melhorar o $R^2$ na base de testes. Use a criatividade, veja se consegue inserir alguma transformação ou combinação de variáveis.
7. Ajuste uma árvore de regressão e veja se consegue um $R^2$ melhor com ela.

In [15]:
df.columns

Index(['Unnamed: 0', 'data_ref', 'index', 'sexo', 'posse_de_veiculo',
       'posse_de_imovel', 'qtd_filhos', 'tipo_renda', 'educacao',
       'estado_civil', 'tipo_residencia', 'idade', 'tempo_emprego',
       'qt_pessoas_residencia', 'mau', 'renda'],
      dtype='object')

In [20]:
df = pd.get_dummies(df,columns=[ 'sexo', 'posse_de_veiculo',
       'posse_de_imovel', 'tipo_renda', 'educacao',
       'estado_civil', 'tipo_residencia','mau'], drop_first=True)

In [27]:
df.tempo_emprego.fillna(0, inplace=True)

In [31]:
X = df.drop(['renda','Unnamed: 0', 'data_ref', 'index'], axis=1)

In [32]:
y = df.renda

In [33]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.25, random_state=42)


## 2.

In [34]:
alphas = [0, 0.001, 0.005, 0.01, 0.05, 0.1]
best_r2_ridge = -1
best_model_ridge = None

In [35]:
for alpha in alphas:
    ridge_model = Ridge(alpha=alpha)
    ridge_model.fit(X_train, y_train)
    y_pred = ridge_model.predict(X_test)
    r2 = r2_score(y_test, y_pred)
    
    if r2 > best_r2_ridge:
        best_r2_ridge = r2
        best_model_ridge = ridge_model


In [36]:
print("Melhor modelo de Regularização Ridge:")
print("R²:", best_r2_ridge)
print("Alpha:", best_model_ridge.alpha)

Melhor modelo de Regularização Ridge:
R²: 0.2362538517163013
Alpha: 0.1


## 3.

In [38]:
best_r2_lasso = -1
best_model_lasso = None


In [39]:
for alpha in alphas:
    lasso_model = Lasso(alpha=alpha)
    lasso_model.fit(X_train, y_train)
    y_pred = lasso_model.predict(X_test)
    r2 = r2_score(y_test, y_pred)
    
    if r2 > best_r2_lasso:
        best_r2_lasso = r2
        best_model_lasso = lasso_model


<ipython-input-39-a959a4601604>:3: UserWarning: With alpha=0, this algorithm does not converge well. You are advised to use the LinearRegression estimator
  lasso_model.fit(X_train, y_train)
C:\Users\TM454YK\Anaconda3\lib\site-packages\sklearn\linear_model\_coordinate_descent.py:530: UserWarning: Coordinate descent with no regularization may lead to unexpected results and is discouraged.
  model = cd_fast.enet_coordinate_descent(
C:\Users\TM454YK\Anaconda3\lib\site-packages\sklearn\linear_model\_coordinate_descent.py:530: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations. Duality gap: 93148182316.422, tolerance: 23680329.459826134
  model = cd_fast.enet_coordinate_descent(
C:\Users\TM454YK\Anaconda3\lib\site-packages\sklearn\linear_model\_coordinate_descent.py:530: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations. Duality gap: 92272956846.57661, tolerance: 23680329.459826134
  model = c

In [40]:
print("Melhor modelo de Regressão LASSO:")
print("R²:", best_r2_lasso)
print("Alpha:", best_model_lasso.alpha)

Melhor modelo de Regressão LASSO:
R²: 0.23628834920885933
Alpha: 0.1


## 4.

In [42]:
estimator = LinearRegression()
rfe_selector = RFE(estimator, n_features_to_select=10)
rfe_selector.fit(X_train, y_train)

RFE(estimator=LinearRegression(), n_features_to_select=10)

In [43]:
X_train_stepwise = rfe_selector.transform(X_train)
X_test_stepwise = rfe_selector.transform(X_test)

In [44]:
stepwise_model = LinearRegression()
stepwise_model.fit(X_train_stepwise, y_train)
y_pred = stepwise_model.predict(X_test_stepwise)
r2_stepwise = r2_score(y_test, y_pred)


In [45]:
print("Modelo Stepwise:")
print("R²:", r2_stepwise)

Modelo Stepwise:
R²: 0.06028119839856849


## 5

In [46]:
print("Coeficientes Ridge:")
print(best_model_ridge.coef_)

print("Coeficientes LASSO:")
print(best_model_lasso.coef_)

print("Coeficientes Stepwise:")
print(stepwise_model.coef_)


Coeficientes Ridge:
[-6.31687097e+02  3.11928015e+01  2.92005706e+02  6.94780092e+02
  2.28686513e+03 -2.76180208e+00  3.88580696e+02 -7.12986417e+02
  9.73470655e+02  1.33391555e+03  5.60061524e+02  2.87854900e+03
  2.55348108e+02  7.64035929e+02  2.74669212e+02  6.67884976e+02
  6.68543914e+02 -1.33933022e+02  6.33587411e+02 -5.61828242e+02
 -7.34775268e+02 -1.13453205e+03 -1.24903832e+03 -1.00292104e+03
  2.64146586e+02]
Coeficientes LASSO:
[-4.51332773e+02  3.12304342e+01  2.91921968e+02  5.14507732e+02
  2.28576204e+03 -2.10614811e+00  3.88308490e+02 -4.49832710e+02
  9.72637853e+02  1.33083729e+03  5.58460946e+02  2.78385610e+03
  2.26061474e+02  7.34114607e+02  2.43166179e+02  4.85351779e+02
  4.89886484e+02 -1.33859036e+02  4.51282207e+02 -5.24089981e+02
 -6.95228127e+02 -1.07368099e+03 -1.19695702e+03 -9.62708218e+02
  2.64685880e+02]
Coeficientes Stepwise:
[1689.84113644  373.58673321  636.87769995 -594.91408706 1344.02986525
 1775.48530168  385.57512034 -988.07273491 -950.93

## 6

In [52]:
X_train['idade_quadratica'] = X_train['idade'] ** 2
X_test['idade_quadratica'] = X_test['idade'] ** 2


<ipython-input-52-9d74b9f6b98b>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  X_train['idade_quadratica'] = X_train['idade'] ** 2
<ipython-input-52-9d74b9f6b98b>:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  X_test['idade_quadratica'] = X_test['idade'] ** 2


In [53]:
model = LinearRegression()
model.fit(X_train, y_train)
y_pred = model.predict(X_test)

r2 = r2_score(y_test, y_pred)
print("R² na base de testes:", r2)

R² na base de testes: 0.23809406633697383


## 7

In [48]:
tree_model = DecisionTreeRegressor()
tree_model.fit(X_train, y_train)
y_pred = tree_model.predict(X_test)
r2_tree = r2_score(y_test, y_pred)


In [49]:
print("Modelo de Árvore de Regressão:")
print("R²:", r2_tree)

Modelo de Árvore de Regressão:
R²: 0.1723174586535472
